In [1]:
import import_ipynb
import utils 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from datetime import datetime


def initialize_webdriver():
    """Initialize and return the WebDriver."""
    driver = webdriver.Chrome()
    return driver

def get_page_data(driver, url):
    """Open the URL and return WebDriverWait instance."""
    driver.get(url)
    return WebDriverWait(driver, 10)

def get_element_text(wait, by, value):
    """Return text content of an element located by the given selector."""
    try:
        element = wait.until(EC.presence_of_element_located((by, value)))
        return element.text.strip()
    except Exception as e:
        print(f"Error locating element {value}: {e}")
        return None

def get_multiple_elements_text(wait, by, value):
    """Return a list of text contents from multiple elements."""
    try:
        elements = wait.until(EC.presence_of_all_elements_located((by, value)))
        return [element.text.strip() for element in elements]
    except (TimeoutException, NoSuchElementException):
        return []  # Return empty list instead of printing error

def extract_tee_time_data(driver, wait, date_value, header_names):
    """Extract and return tee time data from the page."""
    data = []
    try:
        tee_time_rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tr')))
        for row in tee_time_rows:
            try:
                time_value = row.find_element(By.CSS_SELECTOR, 'td.mtrxTeeTimes').text.strip()
                course_value = row.find_element(By.CSS_SELECTOR, 'td.mtrxCourse').text.strip()
                players_value = row.find_element(By.CSS_SELECTOR, 'td.matrixPlayers').text.strip()
                prices = row.find_elements(By.CSS_SELECTOR, 'td.matrixsched div.mtrxPrice, td.matrixsched div.mtrxPriceNA')
                prices_value = [price.text.strip() for price in prices]

                for header, price in zip(header_names, prices_value):
                    data.append((date_value, None, course_value, time_value, players_value, header, price))
            except NoSuchElementException:
                print("Skipping a row due to missing elements.")
                continue  # Skip rows with missing elements
    except TimeoutException:
        print("Timed out waiting for tee time rows to load.")
    return data

def get_foxhollow_tee_time():
    """Main function to scrape data and store it in the database."""
    url = "https://foxhollow.quick18.com/teetimes/searchmatrix?teedate=20250126"
    date_value = datetime.now().strftime('%Y-%m-%d')

    driver = initialize_webdriver()
    wait = get_page_data(driver, url)

    try:
        # Extract header names
        header_names = get_multiple_elements_text(wait, By.CSS_SELECTOR, 'th.matrixHdrSched')
        if not header_names:
            print("No header names found on the page.")
            return []

        # Extract tee time data
        new_data = extract_tee_time_data(driver, wait, date_value, header_names)

        if new_data:
            utils.update_database(new_data, table_name="TeeTimes")
            print("Fox Hollow data has been updated in GolfteeTimes_database.db.")
        else:
            print("No data scraped from Fox Hollow.")
        
        return new_data  # Return the data for use in utils.main()

    except TimeoutException:
        print("Error: Timed out waiting for elements to load on Fox Hollow website.")
        return []  # Return an empty list if scraping fails

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []  # Return an empty list if an unexpected error occurs

    finally:
        driver.quit()  # Ensure the browser is closed even if an error occurs

if __name__ == "__main__":
    utils.initialize_database()
    get_foxhollow_tee_time()


Skipping a row due to missing elements.
Skipping a row due to missing elements.
Skipping a row due to missing elements.
Skipping a row due to missing elements.
Skipping a row due to missing elements.
Skipping a row due to missing elements.
No data scraped from Fox Hollow.
